Tutorial found [here, in the pyTables web site](http://www.pytables.org/usersguide/tutorials.html)

In this notebook, we gonna learn:
* how to save a 3D array into a HDF5 file
* load an array from a HDF5 file


In [2]:
import tables as tb
import numpy as np

ModuleNotFoundError: No module named 'tables'

# Creating a fake 3D data set

In [6]:
stack_images = []
for i in range(10):
    _random_image = np.random.rand(100,100)
    stack_images.append(_random_image)

In [7]:
print(np.shape(stack_images))

(10, 100, 100)


# Saving 3D array into h5 file

In [8]:
output_file = 'my_hdf5_file.h5'

In [9]:
h5file = tb.open_file(output_file, mode='w', title='Test file')

Creating a group

In [ ]:
group = h5file.create_group("/", "MCP", "MCP data set")

creating array object

In [12]:
h5file.create_array(group, "data", stack_images, "3D data set")

/MCP/data (Array(10, 100, 100)) '3D data set'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'python'
  byteorder := 'little'
  chunkshape := None

In [13]:
h5file.close()

# Loading array for h5 file 

In [14]:
input_file = output_file

In [15]:
new_h5file = tb.open_file(input_file, 'a')

In [16]:
print(new_h5file)

my_hdf5_file.h5 (File) 'Test file'
Last modif.: 'Wed Oct 18 12:10:12 2017'
Object Tree: 
/ (RootGroup) 'Test file'
/MCP (Group) 'MCP data set'
/MCP/data (Array(10, 100, 100)) '3D data set'



In [18]:
group = new_h5file.get_node("/MCP/data") 

In [20]:
data = group.read()

In [24]:
print(np.shape(data))
np_data = np.array(data)

(10, 100, 100)


In [25]:
print(type(np_data))

<class 'numpy.ndarray'>


# Looking at metadata inside NeXus 

In [1]:
import os
import h5py

In [2]:
input_data = "/Volumes/G-DRIVE/IPTS/SNAP/SNAP_54081.nxs.h5"
assert os.path.exists(input_data)

In [3]:
f = h5py.File(input_data, 'r')

In [4]:
f.keys()

<KeysViewHDF5 ['entry']>

In [5]:
ipts = str(f['entry']['experiment_identifier'][0])
print(f"ipts: {ipts}")

ipts: b'IPTS-23760'


In [10]:
f['entry']['DASlogs']['BL3:Exp:Det:FileName']['value'][0][0]


b'C:/data/IPTS-23760/Tomorrow_Is_Wednesday/GoodMusic/random_dir/Run_54081_DASTest_vvv_1_000_Stuffb'

In [39]:
mode = f['entry']['DASlogs']['BL3:Exp:Mode']['value_strings'][0][0].decode("utf8")
print(mode)

Imaging Mode


In [27]:
import pathlib

import h5py


def extract_image_directory(nexus_path):
    nexus = pathlib.Path(nexus_path)
    assert nexus.exists()

    with h5py.File(nexus_path, "r") as hdf5_data:
        run_number = hdf5_data["entry"]["entry_identifier"][:][0].decode("utf8")
        pv = hdf5_data["entry"]["DASlogs"]["BL3:Exp:Det:FileName"]
        value = pv["value"][0][0].decode("utf8")
        elements = value.split("/")

        return (
            pathlib.Path("/SNS/SNAP")
            / elements[2]
            / "images"
            / "mcp"
            / "/".join(elements[3:-1])
            / f"Run_{run_number}"
        )
    
def extract_ipts(nexus_path):
    nexus = pathlib.Path(nexus_path)
    assert nexus.exists()

    with h5py.File(nexus_path, "r") as hdf5_data:
        ipts = hdf5_data["entry"]["experiment_identifier"][0].decode("utf8")
        
        return ipts
    
def build_output_directory(input_path, ipts):
    parse_input_path = str(input_path).split("/")
    local_output_path = "/".join(parse_input_path[5:])
    output_folder = f"/SNS/SNAP/{ipts}/shared/autoreduce/{local_output_path}"
    return output_folder

image_directory = extract_image_directory(input_data)
ipts = extract_ipts(input_data)

print(build_output_directory(image_directory, ipts))

/SNS/SNAP/IPTS-23760/shared/autoreduce/mcp/Tomorrow_Is_Wednesday/GoodMusic/random_dir/Run_54081


In [40]:
def extract_instrument_mode(nexus_path):
    nexus = pathlib.Path(nexus_path)
    assert nexus.exists()

    with h5py.File(nexus_path, "r") as hdf5_data:
        instrument_mode = hdf5_data['entry']['DASlogs']['BL3:Exp:Mode']['value_strings'][0][0].decode("utf8")
        
        return instrument_mode

In [41]:
print(extract_instrument_mode(input_data))

Imaging Mode


# Retrieving monitor1 value from NeXus (SNAP) 

In [28]:
import h5py
import os
import json

In [29]:
nexus_path = '/Volumes/G-DRIVE/IPTS/SNAP/SNAP_54282.nxs.h5'
assert os.path.isfile(nexus_path)

In [30]:
f = h5py.File(nexus_path, 'r')

In [31]:
monitor1 = f['entry']['monitor1']['total_counts'][0]
print(f"monitor: {monitor1}")

instrument_mode = f['entry']['DASlogs']['BL3:Exp:Mode']['value_strings'][0][0].decode("utf8")
print(f"instrument mode: {instrument_mode}")

source_filename = f['entry']['DASlogs']['BL3:Exp:Det:FileName']['value'][0]
source_filename = [_entry.decode("utf8") for _entry in source_filename]
print(f"source filename: {source_filename}")

monitor: 7862583
instrument mode: Imaging Mode
source filename: ['C:/data/IPTS-27829/Region_2b_10C/20220225_Run_54281_BM12_0010_0810']


In [32]:
_dict = {'monitor1': int(monitor1),
        'intrument_mode': instrument_mode,
        'source_filename': source_filename}

In [33]:
output_json = "/Users/j35/Desktop/export_json.json"
with open(output_json, 'w') as write_file:
    json.dump(_dict, write_file)

In [34]:
detector_offset = f['entry']['DASlogs']['BL3:Det:dsp1:Trig2:Delay']['average_value'][0]
detector_offset_units = f['entry']['DASlogs']['BL3:Det:dsp1:Trig2:Delay']['average_value'].attrs.get('units').decode("utf8")


print(f"detector_offset: {detector_offset}")
print(f"detector_units: {detector_offset_units}")

detector_offset: 9099832.44175
detector_units: ns


In [44]:
detector_sample_distance = f['entry']['DASlogs']['BL3:Exp:DetectorDistance']['average_value'][0]
detector_sample_units = f['entry']['DASlogs']['BL3:Exp:DetectorDistance']['average_value'].attrs.get('units').decode("utf8")
print(f"detector_sample_distance: {detector_sample_distance}")
print(f"units: {detector_sample_units}")

detector_sample_distance: 20.0
units: m
